<a href="https://colab.research.google.com/github/magotronico/DataAnalysis_and_AI/blob/main/data_science_practice/kpi_per_geo_sucursal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Información de geolocalización por sucursales
Este colab es para poder realizar la liempieza y transformación de los datos y después poder exportarlos a LookerStudio de tal forma que se pueda realizar un mapa de coropletico con los indicadores

Los procesos que se deben de realizar son los siguientes:

*   DB acotada
  1.   Estandarizar los nombres de columnas
  2.   Eliminar caracteres especiales de region, division y sucucursal
  3.   Sumarle a las sucursales que no son BIS sus BIS en caso que tengan

*   DB geolocalizaciones
  1.   Split la columna en 2: [sucursal, ubicacion]
  2.   Agregar el 2ndo nivel de division que necesita LookerStudio para el mapa Coropletico

*   New DataFrame:
  1. hacer un left merge de db_acotada con db_geo en la columna sucursal







## Extract

In [ ]:
import pandas as pd
import numpy as np

# Define files paths
db_acotada_path = "/content/DB's/db_acotada.xlsx"
db_geo_path = "/content/DB's/db_geo.xlsx"

# Read files
db_acotada = pd.read_excel(db_acotada_path)
db_geo = pd.read_excel(db_geo_path, header=None)
db_geo.columns = ["sucursal"]

# # Show info about df
# print("DB ACOTADA")
db_acotada.info()

# print("")

# print("DB GEO")
# print(db_geo.head())
db_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Columns: 116 entries, Region to Quitas Actual
dtypes: float64(113), object(3)
memory usage: 212.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sucursal  202 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


## Transform


### DB Acotada

#### Estandarizacion de columnas

**Explanation:**
* **Lowercase and Underscores:** The lower() method converts all characters to lowercase, and replace() is used to replace spaces and special characters with underscores.

* **Acronyms:** Common phrases are replaced with shorter acronyms (e.g., saldo_insoluto becomes si, vencido becomes vnc, etc.).

* **Time Series Handling:** T-12, T-11, etc., are replaced with m_12, m_11, and so on, to indicate months.

---
**Example Column Renaming:**
* 'Región' → 'region'
* 'Saldo Insoluto T-12' → 'si_m_12'
* 'Saldo Insoluto Actual' → 'si_act'
* 'Pago Anticipado T-12' → 'pa_m_12'
* 'Castigos T-12' → 'cst_m_12'

In [ ]:
def rename_column(col):
    # Replace spaces and special characters
    col = col.lower().replace(' ', '_').replace('\n', '').replace('-', '_').replace('%', 'pct')

    # Create a dictionary for common term replacements
    replacements = {
        'saldo_insoluto': 'si',
        'vencido': 'vnc',
        'actual': 'act',
        'pago_anticipado': 'pa',
        'capital_liquidado': 'cl',
        'capital_dispersado': 'cd',
        'quitas': 'qts',
        'castigos': 'cst'
    }

    # Replace common terms
    for old, new in replacements.items():
        col = col.replace(old, new)

    # Handle time series (T-12, T-11,..., T-01, Actual)
    if '_t_' in col:
        col = col.replace('_t_', '_m_')

    # Map specific time series columns
    column_mapping = {
        'cst_ago_22': 'cst_m_23',
        'cst_sep_22': 'cst_m_22',
        'cst_oct_22': 'cst_m_21',
        'cst_nov_22': 'cst_m_20',
        'cst_dic_22': 'cst_m_19',
        'cst_ene_23': 'cst_m_18',
        'cst_feb_23': 'cst_m_17',
        'cst_mar_23': 'cst_m_16',
        'cst_abr_23': 'cst_m_15',
        'cst_may_23': 'cst_m_14',
        'cst_jun_23': 'cst_m_13'
    }

    # Apply the mapping
    if col in column_mapping:
        col = column_mapping[col]

    # Ensure no double underscores
    col = col.replace('__', '_')

    return col

# Apply renaming function to all columns
db_acotada.columns = [rename_column(col) for col in db_acotada.columns]

# Print new column names
print([col for col in db_acotada.columns])
db_acotada.head()

['region', 'division', 'sucursal', 'si_m_12', 'si_m_11', 'si_m_10', 'si_m_09', 'si_m_08', 'si_m_07', 'si_m_06', 'si_m_05', 'si_m_04', 'si_m_03', 'si_m_02', 'si_m_1', 'si_act', 'si_vnc_m_12', 'si_vnc_m_11', 'si_vnc_m_10', 'si_vnc_m_09', 'si_vnc_m_08', 'si_vnc_m_07', 'si_vnc_m_06', 'si_vnc_m_05', 'si_vnc_m_04', 'si_vnc_m_03', 'si_vnc_m_02', 'si_vnc_m_01', 'si_vnc_act', 'si_30_89_m_12', 'si_30_89_m_11', 'si_30_89_m_10', 'si_30_89_m_09', 'si_30_89_m_08', 'si_30_89_m_07', 'si_30_89_m_06', 'si_30_89_m_05', 'si_30_89_m_04', 'si_30_89_m_03', 'si_30_89_m_02', 'si_30_89_m_01', 'si_30_89_act', 'cst_m_23', 'cst_m_22', 'cst_m_21', 'cst_m_20', 'cst_m_19', 'cst_m_18', 'cst_m_17', 'cst_m_16', 'cst_m_15', 'cst_m_14', 'cst_m_13', 'cst_m_12', 'cst_m_11', 'cst_m_10', 'cst_m_09', 'cst_m_08', 'cst_m_07', 'cst_m_06', 'cst_m_05', 'cst_m_04', 'cst_m_03', 'cst_m_02', 'cst_m_01', 'cst_act', 'cd_m_12', 'cd_m_11', 'cd_m_10', 'cd_m_09', 'cd_m_08', 'cd_m_07', 'cd_m_06', 'cd_m_05', 'cd_m_04', 'cd_m_03', 'cd_m_02', 'c

,region,division,sucursal,si_m_12,si_m_11,si_m_10,si_m_09,si_m_08,si_m_07,si_m_06,...,qts_m_09,qts_m_08,qts_m_07,qts_m_06,qts_m_05,qts_m_04,qts_m_03,qts_m_02,qts_m_01,qts_act
0,Núcleo Uno,División. Red Mexiquense,Ciudad Pirámide,9.329014e+06,1.001554e+07,1.045528e+07,1.095205e+07,1.118846e+07,1.173080e+07,1.216387e+07,...,3825.51,7372.94,1063.22,807.41,10652.32,2149.36,-1864.08,67289.79,2812.35,54416.61
1,Núcleo Uno,División. Red Mexiquense,Valle Verde,1.427773e+07,1.475228e+07,1.518496e+07,1.526722e+07,1.521230e+07,1.570308e+07,1.540092e+07,...,1360.88,7682.34,720.22,-570.84,7774.52,7225.61,18382.39,9666.74,2686.42,1782.76
2,Núcleo Uno,División. Red Mexiquense,Río Blanco,2.487416e+07,2.590822e+07,2.710406e+07,2.811092e+07,2.881454e+07,3.000850e+07,2.984108e+07,...,9684.94,17702.88,53031.55,590.97,28246.12,70887.57,13720.45,10989.27,13063.48,13498.81
3,Núcleo Uno,División. Red Mexiquense,Coli del Sol,2.763131e+07,2.809741e+07,2.809836e+07,2.873883e+07,2.897826e+07,3.029401e+07,2.940537e+07,...,14077.38,47772.82,65789.09,4875.75,1630.34,1519.55,21314.02,7288.87,10574.03,12003.59
4,Núcleo Uno,División. Red Mexiquense,Coli del Sol BIS,2.121196e+05,5.625828e+05,9.251505e+05,1.227236e+06,1.481939e+06,1.800204e+06,2.107983e+06,...,0.00,0.00,0.00,57.90,62.71,8815.17,0.00,324.49,406.83,69.00


#### Estandarizar region, division y sucursal

In [ ]:
import re
import unicodedata

# Function to clean names by removing accents and special characters
def clean_name(name):
    # Normalize the unicode string to decompose the characters
    name = unicodedata.normalize('NFKD', name)
    # Remove accents by taking only the base character
    name = ''.join([c for c in name if not unicodedata.combining(c)])
    # Remove all special characters except alphanumeric and spaces
    name = re.sub(r'[^a-zA-Z0-9\s]', '', name)
    # Remove leading and trailing whitespace
    name = name.strip()
    return name

# Replace wrong parts of sucursals and divisions names
db_acotada['sucursal'] = db_acotada['sucursal'].str.replace('Coli ', 'Colina ', regex=False)
db_acotada['sucursal'] = db_acotada['sucursal'].str.replace('Colis ', 'Colinas ', regex=False)
db_acotada['sucursal'] = db_acotada['sucursal'].str.replace('Lagus ', 'Lagunas ', regex=False)
db_acotada['sucursal'] = db_acotada['sucursal'].str.replace(' Tolá', ' Tonala', regex=False)
db_acotada['sucursal'] = db_acotada['sucursal'].str.replace('Camelis ', 'Camelinas ', regex=False)
db_acotada['sucursal'] = db_acotada['sucursal'].str.replace(' vojoa', ' Navojoa', regex=False)
db_acotada['sucursal'] = db_acotada['sucursal'].str.replace('Lagu ', 'Laguna ', regex=False)
db_acotada['sucursal'] = db_acotada['sucursal'].str.replace('Sabis ', 'Sabinas ', regex=False)
db_acotada['division'] = db_acotada['division'].str.replace('Zo ', 'Zona ', regex=False)

# Remove "BIS" from sucursal and then clean the names
db_acotada['sucursal'] = db_acotada['sucursal'].str.replace(' BIS', '', regex=False)

# Apply the cleaning function to region, division, and sucursal columns
db_acotada["region"] = db_acotada["region"].apply(clean_name)
db_acotada["division"] = db_acotada["division"].apply(clean_name)
db_acotada["sucursal"] = db_acotada["sucursal"].apply(clean_name)

# Group by the cleaned sucursal and sum the specified columns
columns_to_sum = db_acotada.columns[3:]
db_acotada = db_acotada.groupby(['region', 'division', 'sucursal'], as_index=False)[columns_to_sum].sum()

# Save the resulting DataFrame and inspect
db_acotada.to_csv("db_acotada_transformed.csv", index=False)
db_acotada.head()


,region,division,sucursal,si_m_12,si_m_11,si_m_10,si_m_09,si_m_08,si_m_07,si_m_06,...,qts_m_09,qts_m_08,qts_m_07,qts_m_06,qts_m_05,qts_m_04,qts_m_03,qts_m_02,qts_m_01,qts_act
0,Nucleo Dos,Division Distrito Central,Campo Zaragoza,2.631211e+07,2.691944e+07,2.729243e+07,2.738228e+07,2.734799e+07,2.873034e+07,2.797738e+07,...,92767.31,34143.60,6568.35,93151.85,37364.73,4781.56,7592.88,74340.27,15520.21,93744.58
1,Nucleo Dos,Division Distrito Central,Colinas GAM,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,Nucleo Dos,Division Distrito Central,Jardin Aragon A,2.264078e+07,2.375393e+07,2.460094e+07,2.507617e+07,2.498488e+07,2.600885e+07,2.594378e+07,...,6221.11,4638.62,9644.74,9470.70,28591.26,33659.34,26106.43,13250.40,3128.71,32400.23
3,Nucleo Dos,Division Distrito Central,Lomas Zaragoza,5.100963e+06,6.022058e+06,6.587654e+06,7.263507e+06,7.708712e+06,8.331782e+06,8.747468e+06,...,8943.56,-6805.17,1009.09,8973.87,297.31,0.00,3070.31,6603.06,877.14,1612.89
4,Nucleo Dos,Division Distrito Central,Los Arcos,2.797123e+07,2.883083e+07,2.921744e+07,2.903306e+07,2.942890e+07,3.094383e+07,2.959831e+07,...,10072.58,24835.21,9880.33,40455.48,15677.27,1725.14,13966.61,-409.53,29611.76,11949.84


### DB Geolocalizaciones


#### Split gb_geo

In [ ]:
import re

# Define a function to extract latitude and longitude
def extract_lat_long(s):
    match = re.search(r'Latitud:\s*([-\d.]+),\s*Longitud:\s*([-\d.]+)', s)
    if match:
        return f"{float(match.group(1))}, {float(match.group(2))}"
    else:
        return ","
# Apply the function to the 'sucursal' column and create the 'ubicacion' column
db_geo["ubicacion"] = db_geo["sucursal"].apply(extract_lat_long)

db_geo["sucursal"] = db_geo["sucursal"].apply(lambda x: x.split(":")[0])
db_geo.head(10)


,sucursal,ubicacion
0,Ciudad Pirámide,"19.49, -99.23"
1,Valle Verde,"19.52, -99.27"
2,Río Blanco,"19.54, -99.29"
3,Colina del Sol,"19.56, -99.25"
4,Colina del Sol BIS,"19.565, -99.255"
5,Parque Jurica,"19.51, -99.24"
6,Colina Plateada,"19.55, -99.22"
7,Altos de Querétaro,"19.53, -99.21"
8,Sol y Campo,"19.57, -99.2"
9,Satélite 1,"19.5107, -99.2355"


#### Estandarizar las sucursales

In [ ]:
# Eliminar caracteres especiales como acentos, puntos, comas etc
db_geo['sucursal'] = db_geo['sucursal'].apply(clean_name)

# Eliminar las filas que incluyan "BIS" en sucursal
db_geo = db_geo[~db_geo["sucursal"].str.contains("BIS")]
db_geo

,sucursal,ubicacion
0,Ciudad Piramide,"19.49, -99.23"
1,Valle Verde,"19.52, -99.27"
2,Rio Blanco,"19.54, -99.29"
3,Colina del Sol,"19.56, -99.25"
5,Parque Jurica,"19.51, -99.24"
...,...,...
197,Isla del Carmen,"18.64, -91.82"
198,Sierra Merida,"20.97, -89.62"
199,Merida Norte,"21.03, -89.62"
200,Alturas Merida,"21.02, -89.63"


#### Obtener la 2da division de area

Cuando estamos usando el mapa coropletico, necesitamos tener la ubicacion en algun dimensionamiento valido y no en latitud/longitud.

Para esto, se usará la API de Google Geocoding de tal forma que podamos hacer request para cada (lat,lon) y nos regrese el 2ndo nivel de dimensionamiento dentro de un pais (opcion de variable en LookerStudio):

https://developers.google.com/maps/documentation/geocoding/get-api-key?hl=es-419

In [ ]:
import pandas as pd
import requests

# Your Google Maps API key
api_key = 'YourKey'

def reverse_geocode(lat, lon):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}&key={api_key}"
    response = requests.get(url)

    if response.status_code == 200:
        results = response.json().get('results', [])

        if results:
            country, state, city = None, None, None

            for component in results[0]['address_components']:
                # Extract the country
                if 'country' in component['types']:
                    country = component['long_name']

                # Extract the state
                if 'administrative_area_level_1' in component['types']:
                    state = component['long_name']

                # Extract the city or locality
                if 'locality' in component['types']:
                    city = component['long_name']
                elif 'administrative_area_level_2' in component['types']:
                    city = component['long_name']

            # Filter to include only Mexico and USA
            if country in ['Mexico', 'United States']:
                return {
                    'Country': country,
                    'State': state,
                    'City': city
                }

    return {
        'Country': None,
        'State': None,
        'City': None
    }

# Apply the reverse_geocode function to your DataFrame
db_geo[['Country', 'State', 'City']] = db_geo["ubicacion"].apply(
    lambda x: pd.Series(reverse_geocode(
        x.split(',')[0].strip(),  # Extract latitude
        x.split(',')[1].strip()   # Extract longitude
    ))
)

db_geo.head()


,sucursal,ubicacion,Country,State,City
0,Ciudad Piramide,"19.49, -99.23",Mexico,Estado de México,Naucalpan de Juárez
1,Valle Verde,"19.52, -99.27",Mexico,Estado de México,Naucalpan de Juárez
2,Rio Blanco,"19.54, -99.29",Mexico,State of Mexico,Ciudad López Mateos
3,Colina del Sol,"19.56, -99.25",Mexico,Estado de México,Ciudad López Mateos
5,Parque Jurica,"19.51, -99.24",Mexico,Estado de México,Naucalpan de Juárez


In [ ]:
# Save new db_geo
db_geo.to_csv("db_geo_transformed.csv", index=False)

## New DataFrame

### Merge db_acotada and db_geo

In [ ]:
import pandas as pd

# Perform the left merge on the 'sucursal' column
merged_df = pd.merge(db_acotada, db_geo, on='sucursal', how='left')

# List of all columns in merged_df
all_columns = list(merged_df.columns)

# Identify the new columns added from db_geo (excluding 'sucursal')
geo_columns = [col for col in db_geo.columns if col != 'sucursal']

# Reorder columns to ensure geo_columns come right after 'sucursal'
sucursal_index = all_columns.index('sucursal')

# New order: All columns before 'sucursal', 'sucursal', geo_columns, remaining columns
new_order = all_columns[:sucursal_index+1] + geo_columns + [col for col in all_columns if col not in geo_columns and col != 'sucursal' and col != 'region' and col != 'division']

# Apply the new column order to the DataFrame
merged_df = merged_df[new_order]

# Save the result to a CSV file
merged_df.to_csv("merged_db.csv", index=False)

# Display the resulting DataFrame
merged_df.head()


,region,division,sucursal,ubicacion,Country,State,City,si_m_12,si_m_11,si_m_10,...,qts_m_09,qts_m_08,qts_m_07,qts_m_06,qts_m_05,qts_m_04,qts_m_03,qts_m_02,qts_m_01,qts_act
0,Nucleo Dos,Division Distrito Central,Campo Zaragoza,"19.41, -99.175",Mexico,Ciudad de México,Ciudad de México,2.631211e+07,2.691944e+07,2.729243e+07,...,92767.31,34143.60,6568.35,93151.85,37364.73,4781.56,7592.88,74340.27,15520.21,93744.58
1,Nucleo Dos,Division Distrito Central,Colinas GAM,"19.51, -99.12",Mexico,Ciudad de México,Ciudad de México,0.000000e+00,0.000000e+00,0.000000e+00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,Nucleo Dos,Division Distrito Central,Jardin Aragon A,"19.495, -99.11",Mexico,Ciudad de México,Ciudad de México,2.264078e+07,2.375393e+07,2.460094e+07,...,6221.11,4638.62,9644.74,9470.70,28591.26,33659.34,26106.43,13250.40,3128.71,32400.23
3,Nucleo Dos,Division Distrito Central,Lomas Zaragoza,"19.415, -99.18",Mexico,Ciudad de México,Ciudad de México,5.100963e+06,6.022058e+06,6.587654e+06,...,8943.56,-6805.17,1009.09,8973.87,297.31,0.00,3070.31,6603.06,877.14,1612.89
4,Nucleo Dos,Division Distrito Central,Los Arcos,"19.46, -99.145",Mexico,Ciudad de México,Ciudad de México,2.797123e+07,2.883083e+07,2.921744e+07,...,10072.58,24835.21,9880.33,40455.48,15677.27,1725.14,13966.61,-409.53,29611.76,11949.84


### Verify some missing values

In [ ]:
# Check for duplicate column names
print(merged_df.columns[merged_df.columns.duplicated()])

# If there are duplicates, you might want to drop or rename them
# For example, to drop duplicate columns:
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

# Now you can filter for missing 'ubicacion' values
missing_locs = merged_df[merged_df["ubicacion"].isna()]

# Display the resulting DataFrame with missing 'ubicacion'
missing_locs.head()

# If you want to save this result to a CSV file
missing_locs.to_csv("missing_ubicacion.csv", index=False)


Index([], dtype='object')
